# Load AutoLaparo

In [ ]:
import os
import sys
sys.path.append("../..")

import pandas as pd


from utils.io import load_yaml

preview_horizon = 1
seq_len = 15
frame_increment = 5
frames_between_clips = 1

server = "local"
server = load_yaml("../../config/servers.yml")[server]
database = "autolaparo_single_frames/AutoLaparo_Task2"
prefix = os.path.join(server["database"]["location"], database)
# pkl = f"23_03_03_pre_processed_frame_increment_{frame_increment}_frames_between_clips_1_log.pkl"
pkl = f"23_03_03_motion_label_window_1_frame_increment_{frame_increment}_frames_between_clips_1_log_test_train.pkl"

vid_df = pd.read_pickle(os.path.join(prefix, pkl))

# sort!
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(float)
vid_df = vid_df.sort_values(by=["vid", "frame"]).reset_index(drop=True)
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(int)

label_df = pd.read_csv(os.path.join(prefix, "laparoscope_motion_label.csv"))
label_df.Clip = label_df.Clip.apply(lambda x: x-1)

print(vid_df)
print(label_df)

# remove data that is irrelevant to autolaparo
length = len(vid_df[vid_df["vid"] == 0])
print(f"initial length: {length}")

# get last half plus sequence length
# print(len(vid_df)) 250 frames, of which we take half. Increment is missing due to pre-processing
# vid_df = vid_df.groupby("vid").tail(int(length/2) + seq_len - preview_horizon).reset_index(drop=True)
vid_df = vid_df.groupby("vid").tail(125 - frame_increment + seq_len - preview_horizon).reset_index(drop=True)

length = len(vid_df[vid_df["vid"] == 0])
print(f"length after: {length}")

# from readme
motion_dict = {
    0: "Static",
    1: "Up",
    2: "Down",
    3: "Left",
    4: "Right",
    5: "Zoom-in",
    6: "Zoom-out",
}

# Load Homography Predictor

In [ ]:
import torch
from lightning_modules.homography_imitation import ConvHomographyPredictorModule


def cholec80(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/tmp/miccai/final/cholec80/resnet18/version_1"
        checkpoint = "checkpoints/epoch=19-step=39200.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/tmp/miccai/final/cholec80/resnet34/version_1"
        checkpoint = "checkpoints/epoch=19-step=39200.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/tmp/miccai/final/cholec80/resnet50/version_1"
        checkpoint = "checkpoints/epoch=19-step=39200.ckpt"
    return checkpoint_prefix, checkpoint

def heichole(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/tmp/miccai/final/heichole/resnet18/version_0"
        checkpoint = "checkpoints/epoch=40-step=14555.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/tmp/miccai/final/heichole/resnet34/version_1"
        checkpoint = "checkpoints/epoch=34-step=12425.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/tmp/miccai/final/heichole/resnet50/version_0"
        checkpoint = "checkpoints/epoch=41-step=14910.ckpt"
    return checkpoint_prefix, checkpoint

def autolaparo(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/tmp/miccai/final/autolaparo/resnet18/version_0"
        checkpoint = "checkpoints/epoch=26-step=3159.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/tmp/miccai/final/autolaparo/resnet34/version_0"
        checkpoint = "checkpoints/epoch=41-step=4914.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/tmp/miccai/final/autolaparo/resnet50/version_0"
        checkpoint = "checkpoints/epoch=48-step=5733.ckpt"
    return checkpoint_prefix, checkpoint

def phantom(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/tmp/miccai/final/phantom/resnet18/version_0"
        checkpoint = "checkpoints/epoch=138-step=834.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/tmp/miccai/final/phantom/resnet34/version_0"
        checkpoint = "checkpoints/epoch=62-step=378.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/tmp/miccai/final/phantom/resnet50/version_0"
        checkpoint = "checkpoints/epoch=39-step=240.ckpt"
    return checkpoint_prefix, checkpoint

# checkpoint_prefix, checkpoint = cholec80(34)
checkpoint_prefix, checkpoint = heichole(18)
# checkpoint_prefix, checkpoint = phantom(18)

config = load_yaml(os.path.join(checkpoint_prefix, "config.yml"))

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

module = ConvHomographyPredictorModule.load_from_checkpoint(
    os.path.join(checkpoint_prefix, checkpoint), **config["model"]
)
module.to(device)
module = module.eval()
module.freeze()

# Predict Homographies

# Display Images

In [ ]:
from torch.utils.data import DataLoader
from datasets import ImageSequenceMotionLabelDataset, ImageSequenceDataset
import tqdm

from utils.transforms import dict_list_to_augment

import PIL
from IPython.display import display, clear_output
from kornia import tensor_to_image

output_path = "/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_06_trained_model_pred_on_autolaparo"

transforms = dict_list_to_augment([
    {"chance": 1.0, "module": "imgaug.augmenters", "type": "Resize", "kwargs": {"size": {"height": 240, "width": 320}}}
])

ds = ImageSequenceDataset(
    df=vid_df,
    prefix=prefix,
    seq_len=seq_len,
    frame_increment=frame_increment,
    frames_between_clips=seq_len*frame_increment,
    geometric_transforms=transforms,
)

dl = DataLoader(ds, batch_size=1, shuffle=False, num_workers=0)

results = []
for batch in tqdm.tqdm(dl):
    # pre-process
    imgs, imgs_tf, frame_idcs, vid_idcs = batch
    B, T, C, H, W = imgs.shape
    preview_horizon_imgs = imgs[:, :-preview_horizon]
    for img in preview_horizon_imgs[0]:
        display(PIL.Image.fromarray(tensor_to_image(img, keepdim=False)))
        clear_output(wait=True)


## Run Predction

In [ ]:
import os
from torch.utils.data import DataLoader
from datasets import ImageSequenceMotionLabelDataset, ImageSequenceDataset
import tqdm

from utils.transforms import dict_list_to_augment
from utils.viz import create_blend_from_four_point_homography

output_path = "/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_06_trained_model_pred_on_autolaparo"

transforms = dict_list_to_augment([
    {"chance": 1.0, "module": "imgaug.augmenters", "type": "Resize", "kwargs": {"size": {"height": 240, "width": 320}}}
])

delta = 0

ds = ImageSequenceDataset(
    df=vid_df,
    prefix=prefix,
    seq_len=seq_len + delta,
    frame_increment=frame_increment,
    frames_between_clips=20,
    geometric_transforms=transforms,
    random_frame_offset=False
)

dl = DataLoader(ds, batch_size=32, shuffle=False, num_workers=4, drop_last=True)

results = []
for batch in tqdm.tqdm(dl):
    # pre-process
    imgs, imgs_tf, frame_idcs, vid_idcs = batch
    B, T, C, H, W = imgs.shape
    imgs = imgs.to(device).float() / 255.
    preview_horizon_imgs = imgs[:, :-delta-preview_horizon]
    preview_horizon_imgs = preview_horizon_imgs.reshape(B, -1, H, W)
    preview_horizon_imgs = preview_horizon_imgs

    # inference
    duvs = module(preview_horizon_imgs)

    # visualize
    preview_horizon_imgs = preview_horizon_imgs.reshape(B, -1, C, H, W)
    blends = create_blend_from_four_point_homography(preview_horizon_imgs[:, -1], imgs[:, -1], duvs)

    # import PIL
    # from IPython.display import display, clear_output
    # from kornia import tensor_to_image
    # from kornia.geometry import resize
    # import numpy as np
    # for blend in blends:
    #     display(PIL.Image.fromarray((tensor_to_image(resize(blend, [480, 640]), keepdim=False) * 255.).astype(np.uint8)))
    #     clear_output(wait=True)

    # break

    import numpy as np
    duvs_mpd = np.linalg.norm(duvs.cpu().numpy(), axis=-1).mean(axis=-1)

    # logs duvs with video label
    for duv, duv_mpd, vid_idx in zip(duvs, duvs_mpd, vid_idcs):
        # or classify motion here! and keep only directed
        # if duv_mpd > 10: # discard noisy / undecisive motions for binary classification
        label = motion_dict[label_df[label_df["Clip"] == vid_idx.item()]["Label"].values[0]]
        results.append({
            "vid": vid_idx.item(),
            "duv": duv.cpu().numpy().tolist(),
            "label": label,
        })

results_df = pd.DataFrame(results)
name = "_".join(config["experiment"].split("/")[-2:])
results_df.to_pickle(os.path.join(output_path, f"23_03_06_autolaparo_{name}.pkl"))
results_df.to_csv(os.path.join(output_path, f"23_03_06_autolaparo_{name}.csv"))
results_df


# Load and Classify Homographies

# Utilities

In [ ]:
import os
import sys
sys.path.append("../..")

from typing import List
import pandas as pd

from utils.processing import classify_duv_motion

datasets = ["phantom"]
# backbones = ["resnet18", "resnet34", "resnet50"]
backbones = ["resnet18"]

def to_path(datset, backbone, prefix="/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval"):
    return os.path.join(prefix, f"23_03_06_trained_model_pred_on_autolaparo/23_03_06_autolaparo_{datset}_{backbone}.pkl")

# Continuous Classification

In [ ]:
# duv to homography
import numpy as np
import cv2


shape = [240, 320]

# center point
p = np.array([int((shape[0]-1)/2), int((shape[1]-1)/2)]).astype(np.float32)

img_edges = np.array([
    [0, 0],
    [0, shape[1]],
    [shape[0], shape[1]],
    [shape[0], 0],
])

def center_point_classifier(duv: np.array) -> list:
    duv = np.array(row.duv)
    wrp_edges = img_edges + duv
    h = cv2.getPerspectiveTransform(img_edges.astype(np.float32), wrp_edges.astype(np.float32))
    
    # transform center under homography
    p_prime = cv2.perspectiveTransform(p.reshape(-1, 1, 2), h)
    
    # get delta
    duv_center = p_prime - p
    return duv_center[0][0].tolist()

def edge_classifier(duv: np.array) -> list:
    duv = np.array(duv)
    return duv.mean(axis=0).tolist()


for dataset in datasets:
    for backbone in backbones:
        print(backbone)
        df = pd.read_pickle(to_path(dataset, backbone))
        duvs_center = []

        for idx, row in df.iterrows():
            duvs_center.append(center_point_classifier(row.duv))
            # duvs_center.append(edge_classifier(row.duv))

        # log delta p
        df["duv_center"] = duvs_center

        df.to_csv(to_path(dataset, backbone).split(".")[0] + "_duv_center.csv")
        df.to_pickle(to_path(dataset, backbone).split(".")[0] + "_duv_center.pkl")
    break


## Load and Plot

In [ ]:
import os
import sys
sys.path.append("../..")

import pandas as pd

datasets = ["phantom"]
# backbones = ["resnet18", "resnet34", "resnet50"]
backbones = ["resnet18"]

def to_path(datset, backbone, prefix="/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval"):
    return os.path.join(prefix, f"23_03_06_trained_model_pred_on_autolaparo/23_03_06_autolaparo_{datset}_{backbone}_duv_center.pkl")

target_labels = ["Up", "Down", "Left", "Right", "Static"]

for dataset in datasets:
    for backbone in backbones:
        df = pd.read_pickle(to_path(dataset, backbone))
        for label in target_labels:
            sub_df = df[df["label"] == label]
            
            # plot duv scatter
            import matplotlib.pyplot as plt
            plt.scatter(sub_df.duv_center.apply(lambda x: x[1]), sub_df.duv_center.apply(lambda x: -x[0]))
            plt.title(f"{dataset} {backbone} {label}")
            # plt.xlim(-20, 20)
            # plt.ylim(-20, 20)
            plt.show()

            print("x_mean: ", sub_df.duv_center.apply(lambda x: x[1]).mean())
            print("y_mean: ", sub_df.duv_center.apply(lambda x: -x[0]).mean())
        break
    break

# Discrete Classification

In [ ]:
def duv_label() -> List[str]:
    return [f"duv_{i}_{j}" for i in range(4) for j in range(2)]

def split_duv(df: pd.DataFrame) -> pd.DataFrame:
    df_split = df
    df_split["duv"] = df["duv"].apply(
        lambda x: np.array(x).flatten()
    )
    df_split = pd.DataFrame(
        df_split["duv"].to_list(), columns=duv_label()
    )

    df_split = pd.concat(
        [df.reset_index(drop=True), df_split], axis=1
    )

    return df_split

# from readme
motion_dict_other = {
    0: "static",
    1: "up",
    2: "down",
    3: "left",
    4: "right",
    5: "zoom_in",
    6: "zoom_out",
}

true_cnt = 0
false_cnt = 0

for dataset in datasets:
    for backbone in backbones:
        df = pd.read_pickle(to_path(dataset, backbone))
        
        df = split_duv(df)
        df["label"] = df.apply(lambda x: classify_duv_motion(x.duv_0_0, x.duv_0_1, x.duv_1_0, x.duv_1_1, x.duv_2_0, x.duv_2_1, x.duv_3_0, x.duv_3_1, motion_threadhold=7), axis=1)

        for name, group in df.groupby(by="vid"):
            autolaparo_label = motion_dict_other[label_df[label_df["Clip"] == name]["Label"].values[0]]
            labels = group.label.value_counts()
            try:
                labels = labels.drop(index=["static"])
            except:
                pass
            try:
                labels = labels.drop(index=["mixture"])
            except:
                pass
            if len(labels) > 1:
                # print(labels.index[0], autolaparo_label)
                if labels.index[0] == autolaparo_label or labels.index[1] == autolaparo_label: # first most common
                    true_cnt += 1
                else:
                    false_cnt += 1

        print(backbone, true_cnt, false_cnt, true_cnt / (true_cnt + false_cnt))
    break